# Load Data - CS 513 Data Cleaning Final Project

Team 162 - Miffy's Kitchen

In [1]:
import pandas as pd

# Load raw CSVs into DataFrame
parent_folder = 'NYPL-menus'
dish = pd.read_csv(f"{parent_folder}/Dish.csv")
menu = pd.read_csv(f"{parent_folder}/Menu.csv")
menu_item = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page = pd.read_csv(f"{parent_folder}/MenuPage.csv")

# display(dish)
# display(menu)
# display(menu_item)
# display(menu_page)

# *Data Profiling*

In [2]:
dish.describe()

,id,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
count,423397.000000,0.0,423397.000000,423397.000000,423397.000000,423397.000000,394297.000000,394297.000000
mean,264456.594900,NaN,3.060489,3.146794,1675.514555,1679.299738,0.965265,1.603875
std,150489.070889,NaN,27.818178,29.962122,651.321461,651.934580,6.714564,12.696274
min,1.000000,NaN,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000
25%,132374.000000,NaN,1.000000,1.000000,1900.000000,1900.000000,0.000000,0.000000
50%,269636.000000,NaN,1.000000,1.000000,1914.000000,1917.000000,0.000000,0.000000
75%,397135.000000,NaN,1.000000,1.000000,1949.000000,1955.000000,0.400000,0.600000
max,515677.000000,NaN,7740.000000,8484.000000,2928.000000,2928.000000,1035.000000,3050.000000


In [3]:
df = dish
df.info(verbose=True)

columns_with_mv = df.columns[df.isnull().any()]
df[columns_with_mv].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423397 entries, 0 to 423396
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              423397 non-null  int64  
 1   name            423397 non-null  object 
 2   description     0 non-null       float64
 3   menus_appeared  423397 non-null  int64  
 4   times_appeared  423397 non-null  int64  
 5   first_appeared  423397 non-null  int64  
 6   last_appeared   423397 non-null  int64  
 7   lowest_price    394297 non-null  float64
 8   highest_price   394297 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 29.1+ MB


description      423397
lowest_price      29100
highest_price     29100
dtype: int64

## MenuItem Profile

In [12]:
df = menu_item
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332726 entries, 0 to 1332725
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   id            1332726 non-null  int64  
 1   menu_page_id  1332726 non-null  int64  
 2   price         886810 non-null   float64
 3   high_price    91905 non-null    float64
 4   dish_id       1332485 non-null  float64
 5   created_at    1332726 non-null  object 
 6   updated_at    1332726 non-null  object 
 7   xpos          1332726 non-null  float64
 8   ypos          1332726 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 91.5+ MB


In [ ]:
df = menu
# df.info(verbose=True)

fields = ["id", "event"]
null = menu[fields].isnull()
counts = null.sum()

print("res:")
print(menu['event'].isnull().sum()) #/ float(len(menu['event'])))

print("=== Null Count ===")
print(counts)
print("=== Null Percent ===")
print(null.mean())

res:
17545.0
=== Null Count ===
id          0
event    9391
dtype: int64
=== Null Percent ===
id       0.000000
event    0.535252
dtype: float64


In [5]:
# null count as percentage or sum
fields = ["price", "high_price", "menu_page_id"]
null = menu_item[fields].isnull()
counts = null.sum()

print("=== Null Count ===")
print(counts)
print("=== Null Percent ===")
print(null.mean())

=== Null Count ===
price            445916
high_price      1240821
menu_page_id          0
dtype: int64
=== Null Percent ===
price           0.334589
high_price      0.931040
menu_page_id    0.000000
dtype: float64


In [6]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_item
dups = df[df.duplicated(subset=['id'], keep=False)]
print("=== Duplicate Rows ===")
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
print("=== Duplicate Primary Keys ===")
display(bad_df)

pk_violations = bad_df.groupby(['id']).size().to_dict()
print("=== Primary Key Violations ===")
display(pk_violations)

# FK Verification
bad_fk_menu_item_to_menu_page = df[~df["menu_page_id"].isin(menu_page["id"])]
print("=== Foreign Key Violations ===")
print("--- MenuItem to MenuPage - menu_page_id ---")
display(bad_fk_menu_item_to_menu_page)



=== Duplicate Rows ===


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


=== Duplicate Primary Keys ===


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


=== Primary Key Violations ===


{}

=== Foreign Key Violations ===
--- MenuItem to MenuPage - menu_page_id ---


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


In [20]:
# IC Verification - high_price is greater than or equal to price
bad_ic_menu_item_high_price = menu_item[menu_item["high_price"] < menu_item["price"]]
bad_ic_menu_item_high_price

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
1352,1455,168,40.00,0.4,1034.0,2011-04-19 19:40:31 UTC,2011-12-05 03:06:28 UTC,0.217143,0.223982
2122,2303,3079,1.00,0.2,1641.0,2011-04-19 23:38:02 UTC,2011-04-19 23:38:02 UTC,0.172857,0.635264
2464,2694,4782,0.50,0.3,1921.0,2011-04-20 01:39:39 UTC,2011-05-01 04:40:12 UTC,0.408571,0.862274
2489,2724,168,2.00,1.0,1941.0,2011-04-20 01:47:54 UTC,2011-04-20 01:47:54 UTC,0.460000,0.580359
3864,4254,1388,1.00,0.5,2962.0,2011-04-20 06:25:49 UTC,2011-04-20 06:25:49 UTC,0.117143,0.464918
...,...,...,...,...,...,...,...,...,...
1298635,1350803,76350,0.50,0.1,434578.0,2014-09-10 03:58:02 UTC,2014-09-10 03:58:02 UTC,0.526667,0.673693
1301721,1353921,76346,1.00,0.6,1474.0,2014-09-19 14:40:44 UTC,2014-09-19 14:43:02 UTC,0.149333,0.440462
1301724,1353924,76346,50.00,1.0,390145.0,2014-09-19 14:43:19 UTC,2014-09-19 14:43:19 UTC,0.128000,0.569001
1304466,1356727,76544,25.00,5.0,489930.0,2014-10-02 15:06:40 UTC,2014-10-02 15:06:40 UTC,0.412000,0.518781


## MenuPage Profile

In [21]:
# null count as percentage or sum
fields = ["id", "menu_id"]
null = menu_page[fields].isnull()
counts = null.sum()

print("=== Null Count ===")
print(counts)
print("=== Null Percent ===")
print(null.mean())

=== Null Count ===
id         0
menu_id    0
dtype: int64
=== Null Percent ===
id         0.0
menu_id    0.0
dtype: float64


In [22]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_page
dups = df[df.duplicated(subset=['id'], keep=False)]
print("=== Duplicate Rows ===")
display(dups)


=== Duplicate Rows ===


,id,menu_id,page_number,image_id,full_height,full_width,uuid


In [23]:

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
print("=== Duplicate Primary Keys ===")
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
print("=== Primary Key Violations ===")
pk_violations

# FK Verification
bad_fk_menu_page = df[~df["menu_id"].isin(menu["id"])]
print("=== Foreign Key Violations ===")
print("--- MenuPage to Menu - menu_id ---")
display(bad_fk_menu_page)

=== Duplicate Primary Keys ===


,id,menu_id,page_number,image_id,full_height,full_width,uuid


=== Primary Key Violations ===
=== Foreign Key Violations ===
--- MenuPage to Menu - menu_id ---


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99
...,...,...,...,...,...,...,...
66922,77417,35525,2.0,5194206,2942.0,1990.0,af4751c0-6212-0132-6245-58d385a7bbd0
66923,77418,35525,3.0,5194207,2837.0,2628.0,af604d50-6212-0132-b41d-58d385a7bbd0
66924,77419,35525,4.0,5194208,2852.0,2002.0,af7910c0-6212-0132-e227-58d385a7bbd0
66925,77420,35525,5.0,5194209,2835.0,1990.0,af92d3f0-6212-0132-6b8d-58d385a7bbd0


# *OpenRefine Cleaning Verification*

In [22]:
parent_folder = 'NYPL-menus-clean'
# dish_clean = pd.read_csv(f"{parent_folder}/Dish.csv")
menu_clean = pd.read_csv(f"{parent_folder}/menu_clean.csv")
menu_item_clean = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page_clean = pd.read_csv(f"{parent_folder}/MenuPage.csv")

In [25]:
import pandas as pd

# Read in original and cleaned CSV files
df = pd.read_csv(f"NYPL-menus/Menu.csv")
df_clean = pd.read_csv(f"NYPL-menus-clean/menu_clean.csv")

# Count number of cells changed for affected columns
fields = ["event"]
df_clean[df_clean[fields] != df[fields]].count()

id                        0
name                      0
sponsor                   0
event                   999
venue                     0
place                     0
physical_description      0
occasion                  0
notes                     0
call_number               0
keywords                  0
language                  0
date                      0
location                  0
location_type             0
currency                  0
currency_symbol           0
status                    0
page_count                0
dish_count                0
dtype: int64

## Dish Verify

In [11]:
# null count as percentage or sum
fields = ['description']
null = dish[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
description    423397
dtype: int64
=== Percent ===
description    1.0
dtype: float64


In [12]:
# IC Violations and Constraints

# check for duplicate rows
df = dish
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
counts = df.groupby('id')['name'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

ic3_violations = bad_df.groupby(['id', 'name']).size().to_dict()
ic3_violations

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


{}

## MenuItem Verify

In [29]:
# null count as percentage or sum
fields = ["price", "high_price", "menu_page_id"]
null = menu_item_clean[fields].isnull()
counts = null.sum()

print("=== Null Count ===")
print(counts)
print("=== Null Percent ===")
print(null.mean())

=== Null Count ===
price           0
high_price      0
menu_page_id    0
dtype: int64
=== Null Percent ===
price           0.0
high_price      0.0
menu_page_id    0.0
dtype: float64


In [27]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_item_clean
dups = df[df.duplicated(subset=['id'], keep=False)]
print("=== Duplicate Primary Keys ===")
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id']).size().to_dict()
print("=== Primary Key Violations ===")
display(pk_violations)

# FK Verification
bad_fk_menu_item_to_menu_page = df[~df["menu_page_id"].isin(menu_page["id"])]
print("=== Foreign Key Violations ===")
print("--- MenuItem to MenuPage - menu_page_id ---")
display(bad_fk_menu_item_to_menu_page)

=== Duplicate Primary Keys ===


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


=== Primary Key Violations ===


{}

=== Foreign Key Violations ===
--- MenuItem to MenuPage - menu_page_id ---


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


In [28]:
# IC Verification - high_price is greater than or equal to price
bad_ic_menu_item_high_price = menu_item_clean[menu_item_clean["high_price"] < menu_item_clean["price"]]
bad_ic_menu_item_high_price

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


## MenuPage Verify

In [15]:
# null count as percentage or sum
fields = ["id", "menu_id"]
null = menu_page_clean[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
id         0
menu_id    0
dtype: int64
=== Percent ===
id         0.0
menu_id    0.0
dtype: float64


In [16]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_page_clean
dups = df[df.duplicated(subset=['id'], keep=False)]
print("=== Duplicate Rows ===")
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
print("=== Duplicate Primary Keys ===")
display(bad_df)

pk_violations = bad_df.groupby(['id']).size().to_dict()
print("=== Primary Key Violations ===")
pk_violations

# FK Verification
bad_fk_menu_page = df[~df["menu_id"].isin(menu["id"])]
print("=== Foreign Key Violations ===")
print("--- MenuPage to Menu - menu_id ---")
display(bad_fk_menu_page)

=== Duplicate Rows ===


,id,menu_id,page_number,image_id,full_height,full_width,uuid


=== Duplicate Primary Keys ===


,id,menu_id,page_number,image_id,full_height,full_width,uuid


=== Primary Key Violations ===
=== Foreign Key Violations ===
--- MenuPage to Menu - menu_id ---


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99
...,...,...,...,...,...,...,...
66922,77417,35525,2.0,5194206,2942.0,1990.0,af4751c0-6212-0132-6245-58d385a7bbd0
66923,77418,35525,3.0,5194207,2837.0,2628.0,af604d50-6212-0132-b41d-58d385a7bbd0
66924,77419,35525,4.0,5194208,2852.0,2002.0,af7910c0-6212-0132-e227-58d385a7bbd0
66925,77420,35525,5.0,5194209,2835.0,1990.0,af92d3f0-6212-0132-6b8d-58d385a7bbd0


## Menu Verify

In [24]:
df = menu_clean
# df.info(verbose=True)

fields = ["event"]
null = menu[fields].isnull()
counts = null.sum()

print("res:")
print(menu['event'].isnull().sum()) #/ float(len(menu['event'])))

res:
9391


# Data Cleaning

## Foreign Key Violations - MenuItem

In [17]:
# Since we cannot reconstruct what menu_page_id should be from within MenuItem, we must delete these rows
menu_item_clean = menu_item_clean[~menu_item_clean["id"].isin(bad_fk_menu_item_to_menu_page["id"])]

# Verify all foreign key constraints have been resolved
display(menu_item_clean[~menu_item_clean["menu_page_id"].isin(menu_page["id"])])



,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


In [18]:
# Since we cannot reconstruct what menu_id should be within MenuPage, we must delete these rows
menu_page_clean = menu_page_clean[~menu_page_clean["id"].isin(bad_fk_menu_page["id"])]

# Verify all foreign key constraints have been resolved
display(menu_page_clean[~menu_page_clean["menu_id"].isin(menu["id"])])



,id,menu_id,page_number,image_id,full_height,full_width,uuid


In [ ]:
# save cleaned csvs

parent_folder = "NYPL-menus-clean"
menu_item_clean.to_csv(f"{parent_folder}/menu_item_clean.csv")
menu_page_clean.to_csv(f"{parent_folder}/menu_page_clean.csv")